### 作業目的: 使用N-Gram模型預測文字

本次作業會使用[桃園市官網市政新聞](https://data.gov.tw/dataset/25891)來進行練習

### 載入所需的Libraries

In [1]:
import json
import re 
from collections import Counter, namedtuple

### 載入資料

In [2]:
with open('./WebNews.json', 'r', encoding = 'utf-8') as f:
    data = f.read()

In [3]:
news_data = json.loads(data)

In [4]:
news_data[:2]

[{'file': [],
  'link': [],
  'id': '202006110003',
  'istop': 'N',
  'img': [{'imgname': '0611桃園屏東_200611_0004.jpg',
    'imgcontent': '<br>鄭<br>市<br>長<br>與<br>潘<br>縣<br>長<br>合<br>體<br>推<br>廣<br>防<br>疫<br>互<br>惠<br>旅<br>遊<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817400.jpg&filedisplay=202006111817400.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0005.jpg',
    'imgcontent': '<br>潘<br>縣<br>長<br>挑<br>選<br>屏<br>東<br>特<br>色<br>伴<br>手<br>禮<br>致<br>贈<br>鄭<br>市<br>長<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817401.jpg&filedisplay=202006111817401.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0007.jpg',
    'imgcontent': '<br>鄭<br>市<br>長<br>精<br>心<br>準<br>備<br>桃<br>園<br>特<br>色<br>伴<br>手<br>禮<br>送<br>給<br>潘<br>縣<br>長<br>',
    'imgurl': 'http://www.tycg.gov.tw/uploaddowndoc?file=news/202006111817402.jpg&filedisplay=202006111817402.jpg&flag=pic'},
   {'imgname': '0611桃園屏東_200611_0006.jpg',
    'imgcontent': '<br>桃<br>園<br

### 進行資料清洗
觀察上面的資料，資料包含許多其他的資訊，我們需要將真正的新聞內文取出，並且對內文進行文字清洗。
請做以下的文字處理:

1. 取出新聞內文
2. 去除HTML Tags
3. 移除標點符號，只保留英文、數字、中文

In [5]:
#取出新聞內文
#去除HTML Tags與標點符號(只保留英文、數字、中文)

corpus_list = []
for i in range(len(news_data)):
    pattern = r'(?<=\'detailcontent\':\s\'\<strong\>).*(?=\'secsubject\')'
    re_text = re.findall(pattern,str(news_data[i]),re.M)
    pattern = re.compile('(<.*?>)|(&nbsp;)')
    re_text2 = re.sub(pattern, '', str(re_text))
    pattern = r'[\u4e00-\u9fa5_a-zA-Z]+'
    corpus_list.append(''.join((re.findall(pattern, re_text2, re.M))))
    
print(corpus_list[0])

迎接國旅爆發期五星縣市長合體推廣桃園屏東互惠旅遊桃園市長鄭文燦今日下午出席桃園加屏東旅遊優惠強棒一棒接一棒好康發表記者會鄭市長表示隨著疫情趨緩國內進入國旅爆發期觀光局近期公布最新大熱門旅遊景點桃園包辦前三名依序為大溪老街石門水庫及角板山顯示桃園旅遊的潛力無限鄭市長說桃園市與屏東縣各自擁有獨特的觀光魅力與資源希望能搭配中央觀光振興方案共同推廣防疫互惠旅遊並推出更多加碼優惠措施降低民眾的旅遊負擔增加旅遊的樂趣也讓國境之南與國境之門做最好的交流鄭市長提到桃園是國門之都屏東是國境之南兩個城市各有特色及魅力許多景點也互相輝映假如你喜歡屏東的山川琉璃吊橋絕對不能錯過桃園的新溪口吊橋如果你喜歡大溪老街也應該去恆春古城漫遊鄭市長說許多民眾造訪過很多國外景點例如普吉島巴里島卻沒有到過小琉球澎湖等離島十分可惜他也稱讚屏東幅員遼闊擁有多樣化的景點墾丁更是國內旅遊勝地之一希望現階段推動防疫新生活的同時桃園也能與各縣市合作一起推廣防疫互惠旅遊讓更多在地旅遊業者受惠桃園預估下半年包含鄰里長環保志工守望相助隊義警民防等觀摩旅遊團數出團數將達團以上目前桃園第一階段已經與基隆台南屏東等縣市簽署合作契約第二階段將與宜蘭花蓮南投嘉義等縣市合作希望未來逐漸擴大至其他縣市提供遊客更多優惠措施鄭市長也說屏東縣推出三鐵優惠方案桃園市自月日起推出來桃園住一晚加贈免費一日遊方案遊客來桃園在合法旅宿住一晚即可獲得市府加贈免費一日遊行程內容包含遊覽車接送午餐景點門票或DIY體驗等期盼帶領遊客體驗桃園的觀光魅力此外民眾也可以持元振興三倍券兌換元夜市券藉此鼓勵民眾到夜市消費市府未來也將陸續推出更多的加碼優惠措施屏東縣長潘孟安則希望在鄭市長推廣下吸引更多桃園市民到屏東旅遊他表示桃園為國境之門擁有很大的發展優勢在鄭市長帶領下桃園連續多年結婚率及出生率都是全國之冠也成為一座宜居城市他也特別推薦親子旅遊首選屏東屏東縣政府整合轄內旅宿業者伴手禮業者食品業者等推出鐵定加碼鐵定貼心鐵定好玩三鐵優惠月日以前遊客在恆春地區住宿一晚即加贈場館門票或等值餐飲券在東港地區住宿則可用元加購伴手禮及小王馬福袋人以上團體旅遊每人可獲得元旅費補助每團上限萬元屏東也有許多當季優質農特產品及特色伴手禮歡迎桃園市民踴躍到屏東旅遊保證遊客玩得痛快買得愉快吃得爽快全國中等學校運動會今年首度移師屏東舉辦潘縣長也特別邀請鄭市長在月日開幕式當天帶領市府團隊一同到屏東為桃

### 建立N-Gram模型
N-Gram模型在計算詞機率時為(以Trigram為例)
$$
P(W_i|W_{i-1},W_{i-2}) = \frac{count(W_i,W_{i-1},W_{i-2})}{count(W_{i-1},W_{i-2})}
$$

舉例來說
$$
P(the|this,is) = \frac{count(this\ is\ the)}{count(this\ is)}
$$

In [6]:
def ngram(documents, N=4):
    
    #建立儲存預測字, 所有ngram詞頻字典, 所有字詞(分母)
    gram_frequency = dict()
    ngram_prediction = dict()
    total_grams = list()
    words = list()
    gram_key = dict()
    gram_key_prob = dict()

    for doc in documents:
        # 在每個文章前加上起始(<start>)與結束符號(<end>)
        word_list = ["<s>"] + list(doc) + ["<e>"]
        
        # 計算分子
        for i in range(len(word_list) - (N-1)):
            total_grams.append(tuple(word_list[i:i+N]))           
        
        # 計算分母        
        for i in range(len(word_list) - (N-2)):
            words.append(tuple(word_list[i:i+N-1])) 
    
    #計算分子詞頻
    total_word_counter = Counter(total_grams)
    #計算分母詞頻
    word_counter = Counter(words)
    
    #計算所有N-gram預測字詞的機率
    four_gram_freq = sorted(total_word_counter.items(), key = lambda word_count: word_count[1], reverse=True)
    for key in four_gram_freq:    
        gram_key[key[0][:3]] = gram_key.get(key[0][:3],0) + key[1]    

    for key2 in four_gram_freq:
        if key2[0][:3] not in gram_key_prob.keys():
            gram_key_prob[key2[0][:3]] = []
            gram_key_prob[key2[0][:3]] = [[key2[0][3],key2[1]/gram_key[key2[0][:3]]]]
        else:
            gram_key_prob[key2[0][:3]] = gram_key_prob[key2[0][:3]]+[[key2[0][3],key2[1]/gram_key[key2[0][:3]]]]
        
    return gram_key_prob

### 使用N-Gram模型進行預測
這裡我們使用4 gram模型，也就是輸入三個字之後，第四個字出現的機率，並將輸出依據機率做排序

In [7]:
#建立four_gram模型，並將預測的機率按照大小排列
four_gram_pred = ngram(corpus_list)
#for word, pred in four_gram_pred.items():
four_gram_pred

{('市', '長', '表'): [['示', 0.9968911917098445], ['揚', 0.0031088082901554403]],
 ('鄭', '市', '長'): [['表', 0.2705577918343876],
  ['也', 0.21995399654974124],
  ['說', 0.12478435882691202],
  ['指', 0.12047153536515239],
  ['提', 0.07245543415756182],
  ['回', 0.03220241518113859],
  ['談', 0.015238642898217367],
  ['在', 0.01408855664174813],
  ['感', 0.012075905692926969],
  ['補', 0.010638297872340425],
  ['強', 0.010350776308223116],
  ['並', 0.006612995974698103],
  ['進', 0.006612995974698103],
  ['特', 0.005462909718228867],
  ['則', 0.00488786658999425],
  ['於', 0.0040253018976423235],
  ['期', 0.0037377803335250145],
  ['的', 0.002875215641173088],
  ['肯', 0.002875215641173088],
  ['亦', 0.002587694077055779],
  ['今', 0.0023001725129384704],
  ['視', 0.0023001725129384704],
  ['慰', 0.0020126509488211618],
  ['認', 0.0020126509488211618],
  ['祈', 0.0020126509488211618],
  ['請', 0.0017251293847038527],
  ['除', 0.0017251293847038527],
  ['與', 0.001437607820586544],
  ['分', 0.001437607820586544],
  ['祝',

In [8]:
Word = namedtuple('Word', ['word', 'prob']) #使用namedtuple來儲存預測字詞與對應機率
words = [''.join(x) for x in four_gram_pred.keys()]
prob = [y for y in four_gram_pred.values()]
prob_namedtuple = []
for x in prob:
    prob_namedtuple = prob_namedtuple + ([[Word(y[0],y[1]) for y in x]])

In [9]:
four_gram_pred_Word = dict(zip(words, prob_namedtuple))
four_gram_pred_Word

{'市長表': [Word(word='示', prob=0.9968911917098445),
  Word(word='揚', prob=0.0031088082901554403)],
 '鄭市長': [Word(word='表', prob=0.2705577918343876),
  Word(word='也', prob=0.21995399654974124),
  Word(word='說', prob=0.12478435882691202),
  Word(word='指', prob=0.12047153536515239),
  Word(word='提', prob=0.07245543415756182),
  Word(word='回', prob=0.03220241518113859),
  Word(word='談', prob=0.015238642898217367),
  Word(word='在', prob=0.01408855664174813),
  Word(word='感', prob=0.012075905692926969),
  Word(word='補', prob=0.010638297872340425),
  Word(word='強', prob=0.010350776308223116),
  Word(word='並', prob=0.006612995974698103),
  Word(word='進', prob=0.006612995974698103),
  Word(word='特', prob=0.005462909718228867),
  Word(word='則', prob=0.00488786658999425),
  Word(word='於', prob=0.0040253018976423235),
  Word(word='期', prob=0.0037377803335250145),
  Word(word='的', prob=0.002875215641173088),
  Word(word='肯', prob=0.002875215641173088),
  Word(word='亦', prob=0.002587694077055779),
  W

In [10]:
text = '鄭文燦' 
four_gram_pred[text[0],text[1],text[2]]

[['今', 0.9814207650273225],
 ['市', 0.007650273224043716],
 ['模', 0.002185792349726776],
 ['表', 0.002185792349726776],
 ['台', 0.001092896174863388],
 ['率', 0.001092896174863388],
 ['於', 0.001092896174863388],
 ['昨', 0.001092896174863388],
 ['成', 0.001092896174863388],
 ['回', 0.001092896174863388]]

可自行嘗試使用不同的N搭建不同的N-Gram模型的效果